In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from drcomp.reducers import PCA
from drcomp import estimate_intrinsic_dimension
import pandas as pd

In [ ]:
dorothea = pd.read_csv("../data/raw/dorothea.csv", header=0, sep=",")
dorothea.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
dorothea.head()

In [ ]:
dorothea.shape

In [ ]:
dim = estimate_intrinsic_dimension(dorothea)
model = PCA(n_components=dim).fit(dorothea)

In [ ]:
Y = pd.DataFrame(model.transform(dorothea))

In [ ]:
Y.head()

In [ ]:
model.pca.explained_variance_ratio_.sum()

In [ ]:
model.evaluate(dorothea)

In [ ]:
model.drm_.report()